#### Ai Agents in LangGraph Course

In [1]:
import os
import re
from pathlib import Path
from dotenv import load_dotenv

from langchain_groq import ChatGroq

# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated # to construct the agent's state
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add] # operator.add allows us to add messages instead of replacing them when the LLM's output is returned

In [4]:
# # Used for persistence
# from langgraph.checkpoint.sqlite import SqliteSaver
# memory = SqliteSaver.from_conn_string(":memory:")

In [9]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState) # initialize graph

        # Add nodes
        graph.add_node("llm", self.call_groq)
        graph.add_node("action", self.take_action)
        
        # The edge where the decision to use a tool is made
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        
        # Create edge and set starting point of the graph
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")

        self.graph = graph.compile() # Build graph
        self.tools = {t.name: t for t in tools} # Save the tools' names that can be used
        self.model = model.bind_tools(tools) # Provide the name of the tools to the agent

    # Tells the agent if action is needed by checking the last message in the state which is supposed to contain this info
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    # Call the LLM and update the Agent's State by adding the response
    def call_groq(self, state: AgentState):
        messages = state['messages']
        if self.system: messages = [SystemMessage(content=self.system)] + messages
        
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # Search for the tool and use it
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

        print("Back to the model!")
        return {'messages': results}

In [6]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")
abot = Agent(model, [tool], system=prompt)

In [ ]:
# ## Visualize Graph
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [12]:
# Results may vary per run and over time as search information and models change.
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_vha9', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'headquarters location of [insert winning team name here]'}, 'id': 'call_ty0c', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of [insert state name here]'}, 'id': 'call_p58m', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_c3rb', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri'}, 'id': 'call_09ge', 'type': 'tool_call'}
Back to the model!
The Kansas City Chiefs won the Super Bowl in 2024. The team's headquarters is located in Kansas City, Missouri. The GDP of Missouri was $336.63 billion in 2022, a 2.0 percent increase from 2021 to 2022.


### Persistence